In [1]:
from directory_cobveg_treat import definir_diretorio
project_path=definir_diretorio()

import os
import geopandas as gpd

from utils import (
    get_data_diretorio,
    save_parquet_excel
)

# Cobertura Vegetal por habitante (m²/hab)

In [2]:
data_path = get_data_diretorio(project_path)

# Dependências

Este notebook é dependente do asset resultante do notebook '[cebrap]cobertura_vegetal_por_distrito'

In [3]:
dist_cobveg_path = os.path.join(
    data_path,
    'assets',
    'cobertura_vegetal', 
    'distritos_cobertura_vegetal.parquet'
)
gdf_dist_cobveg = gpd.read_parquet(
    dist_cobveg_path
)

?? -> o resultado de gdf['geometry'].area é m² mesmo, né?

# Cálculo per capita (m²/hab)

In [4]:
gdf_dist_cobveg.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,...,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg
25,3,Sudeste,35,São Paulo,3550308,São Paulo,355030826,CONSOLACAO,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,3.786948,53144,35777,"POLYGON ((331116.481 7395552.671, 331137.408 7...",1.049232e+06,3.786605e+06,0.277090
0,3,Sudeste,35,São Paulo,3550308,São Paulo,355030801,AGUA RASA,3501,São Paulo,...,São Paulo,3550308,São Paulo/SP,7.134883,85788,41161,"POLYGON ((339574.362 7391448.736, 339568.669 7...",6.492151e+05,7.133711e+06,0.091007


In [5]:
gdf_dist_cobveg['cob_veg_per_capita'] = (
    gdf_dist_cobveg['area_dist_cobveg']/gdf_dist_cobveg['total_pop']
)

In [6]:
gdf_dist_cobveg.sample(2)

,CD_REGIAO,NM_REGIAO,CD_UF,NM_UF,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,...,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,area_dist_cobveg,area_distrito,pct_dist_cobveg,cob_veg_per_capita
23,3,Sudeste,35,São Paulo,3550308,São Paulo,355030824,CIDADE LIDER,3501,São Paulo,...,3550308,São Paulo/SP,10.558185,136660,54530,"POLYGON ((347490.062 7391594.735, 347468.66 73...",2.272720e+06,1.055578e+07,0.215306,16.630469
67,3,Sudeste,35,São Paulo,3550308,São Paulo,355030868,SACOMA,3501,São Paulo,...,3550308,São Paulo/SP,14.610797,261436,113894,"POLYGON ((338605.158 7385601.06, 338598.678 73...",2.575763e+06,1.460870e+07,0.176317,9.852364


In [7]:
sum(gdf_dist_cobveg['cob_veg_per_capita'].isnull())

0

Como precisamos saber qual a unidade de medida, vamos confirmar as informações do crs:

In [8]:
gdf_dist_cobveg.crs

<Projected CRS: EPSG:31983>
Name: SIRGAS 2000 / UTM zone 23S
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Brazil - between 48°W and 42°W, northern and southern hemispheres, onshore and offshore.
- bounds: (-48.0, -33.5, -42.0, 5.13)
Coordinate Operation:
- name: UTM zone 23S
- method: Transverse Mercator
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

Segundo as informações da projeção, a unidade é em metros e não há a necessidade de alterar.

In [9]:
max_pct=gdf_dist_cobveg['cob_veg_per_capita'].max()
max_name=gdf_dist_cobveg['NM_DIST'].loc[gdf_dist_cobveg['cob_veg_per_capita']==max_pct].iloc[0]
print(f'{max_name} é o distrito com a maior área de cobertura vegetal per capita: {round (max_pct, 2)} m²/hab')

MARSILAC é o distrito com a maior área de cobertura vegetal per capita: 17598.75 m²/hab


In [10]:
min_pct=gdf_dist_cobveg['cob_veg_per_capita'].min()
min_name = gdf_dist_cobveg['NM_DIST'].loc[gdf_dist_cobveg['cob_veg_per_capita']==min_pct].iloc[0]
print(f'{min_name} é o distrito com a menor porcentagem de cobertura vegetal: {round(min_pct, 2)} m²/hab')

REPUBLICA é o distrito com a menor porcentagem de cobertura vegetal: 4.61 m²/hab


# Salvar gdf

In [11]:
save_parquet_excel(
    gdf_dist_cobveg, 
    fname= 'cobertura_vegetal_per_capita',
    data_path = data_path,
    data_subpath = os.path.join(
        'assets',
        'cobertura_vegetal'
    )
)